# 02_evaluation_report — 固定パイプラインで一発評価

`assets/fixed_pipeline.json` を読み込み、eval カテゴリ（例：leather, tile）に対して、
train_normal のみで fit → test を一度だけ評価し、指標・図表を保存するテンプレートです。
リーク防止のため、本ノートではパラメータ・閾値を変更しません。


## 依存と設定の読み込み

In [ ]:
import os
from pathlib import Path
import json
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ASSETS = Path("assets")
RUNS = Path("runs")
RUNS.mkdir(parents=True, exist_ok=True)
CFG = ASSETS / "fixed_pipeline.json"
if not CFG.exists():
    raise FileNotFoundError("assets/fixed_pipeline.json が見つかりません。まず 01_experiments_dev.ipynb を完了してください。")

with CFG.open("r", encoding="utf-8") as f:
    fixed = json.load(f)
fixed


## データ取得（anomalib 経由）

In [ ]:
# AGENTS.md: anomalib によるデータ取得。既存の MVTEC_ROOT / data/mvtec を尊重。
MVTEC_ROOT = Path(os.environ.get("MVTEC_ROOT", "data/mvtec"))
MVTEC_ROOT.mkdir(parents=True, exist_ok=True)
try:
    from anomalib.data import MVTecAD
    _ = MVTecAD(root=str(MVTEC_ROOT))
except Exception as e:
    print("[WARN] anomalib セットアップで問題が発生しました。", e)


## 評価カテゴリの選択と実行（テンプレート）
- ここで eval カテゴリ（例：leather, tile）を選び、固定パイプラインで評価します。
- 本テンプレートではダミー実装で runs/eval/** に metrics.json を書き出す骨子のみ用意します。

In [ ]:
eval_categories = ["leather", "tile"]  # 必要に応じて拡張

def evaluate_category(category: str, fixed: dict):
    """テンプレートのダミー評価関数。
    実装時は fixed の設定を用いて train_normal で fit → test を一度だけ評価してください。
    戻り値: 指標 dict（AUROC/AUPRC/F1 など）
    """
    # TODO: 実装。ここではダミー値を返す。
    return {"auroc": None, "auprc": None, "f1": None}

from datetime import timezone
for cat in eval_categories:
    metrics = evaluate_category(cat, fixed)
    out_dir = RUNS / "eval" / cat
    out_dir.mkdir(parents=True, exist_ok=True)
    with (out_dir / "metrics.json").open("w", encoding="utf-8") as f:
        json.dump({"category": cat, "metrics": metrics, "timestamp": datetime.now(timezone.utc).isoformat()}, f, indent=2, ensure_ascii=False)
    print(f"[INFO] Wrote metrics: {out_dir / 'metrics.json'}")


## 図表の作成・保存（テンプレート）
- ROC/PR カーブや失敗例サムネイルを `assets/figs/**` に保存する処理を実装してください。
- 本テンプレートでは出力先のみ用意します。

In [ ]:
FIGS = Path("assets/figs")
FIGS.mkdir(parents=True, exist_ok=True)
print(f"[INFO] Figure dir: {FIGS.resolve()}")


## README 連携（メモ）
- 最終的に README のサマリー表・図を更新してください（半自動でも可）。
